# Integrate Our Separate Data Streams into One Data Frame

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
intro = "../data/"
tweets = pd.read_pickle(intro+"02_geotweets_labeled_ee.pkl")
curpop = pd.read_csv(intro+"curpop_df.csv", index_col = 0)
weekpop = pd.read_csv(intro+"weekpop_df.csv", index_col = 0)
img = pd.read_csv(intro+"image_label.csv", index_col = 0)
curlog = pd.read_csv(intro+"current_log.csv")
wlog = pd.read_csv(intro+"binned_weather_history.csv")
survey = pd.read_csv(intro+"Survey_clean.csv")

### Obtain a Data Frame of Time Bins Labeled by Tweet Labels

In [3]:
# tweets.head()

,date,text,username,day_of_week,hour,safe,time_bin
0,2020-03-23 13:57:44-04:00,Omg @Instacart - y’all better not mess this up...,JordanGazay,Monday,13,NaN,2020-03-23 13:50:00
1,2020-03-23 14:10:14-04:00,𝐐𝐔𝐄𝐒𝐓𝐈𝐎𝐍 | Are you moving for thirty minutes e...,lifenleggings,Monday,14,0.0,2020-03-23 14:05:00
2,2020-03-23 15:03:32-04:00,"#newnormal Day 8 Oh dayum, did Governor #andre...",miles_randy,Monday,15,NaN,2020-03-23 14:50:00
3,2020-03-23 18:14:17-04:00,#isolation #prospectpark #prospectparkalliance...,andrewbwhite1,Monday,18,NaN,2020-03-23 18:05:00
4,2020-03-23 18:15:25-04:00,#isolation #prospectpark #fishing #prospectpar...,andrewbwhite1,Monday,18,NaN,2020-03-23 18:05:00


In [4]:
# Note: in some cases, we have multiple tweets for a single time bin.
# In what follows, it'll make life easier if we swap -1 for 2 as an indicator of 
# no information. We will later swap 2 back for -1 to match everyone else. 
safe = []
for i in range(0,len(tweets)):
    value = tweets.iloc[i].safe
    if value == 2:
        safe.append(-1)
    else:
        safe.append(value)

tweets['safe'] = safe

bin_counts = tweets.groupby('time_bin')['safe'].agg('count')
bin_counts
bigbins = bin_counts[bin_counts > 1]
bigbins = bigbins.reset_index()
bigbin_tweets = pd.merge(left = bigbins.time_bin, right = tweets, left_on = ['time_bin'], right_on = ['time_bin'])

# Now we can check manually whether we have any bins containing tweets with polar opposite labels
# as opposed to either 0 or 1 + uninformative.
# Note: this is a substantive decision, since it means that if some tweets are informative but others not, we 
# label the tweet according to the information we got from the informative tweets, rather than ignoring this 
# information. 

for i in range(0,len(bigbin_tweets)-1):
    if bigbin_tweets.iloc[i].safe + bigbin_tweets.iloc[i+1].safe == 1:
        print(i, ":", bigbin_tweets.iloc[i], i+1, ":", bigbin_tweets.iloc[i+1])

In [5]:
# Again, here we are checking to see whether we have any adjacent rows with opposite labels: 
# 0 next to 1 or vice versa.
# There are none. So we can safely assign each bin the max of the safe values that occur in that bin.
tweet_bin_labels = tweets.groupby('time_bin')['safe'].agg('max')
# This is why we changed NaN to -1. 

In [6]:
tweet_bin_labels = tweet_bin_labels.reset_index()
# tweet_bin_labels

,time_bin,safe
0,2020-03-23 13:50:00,-1.0
1,2020-03-23 14:05:00,0.0
2,2020-03-23 14:50:00,-1.0
3,2020-03-23 18:05:00,-1.0
4,2020-03-24 08:35:00,-1.0
...,...,...
194,2020-04-14 13:20:00,-1.0
195,2020-04-14 15:20:00,-1.0
196,2020-04-14 16:50:00,-1.0
197,2020-04-14 18:35:00,-1.0


In [7]:
# Standardize interpretation of labels: 0 = safe, 1 = unsafe, 2 = unsure
# Add column indicating if it's a weekend (1) or not (0)

safe = []
weekend = []

for i in range(0,len(tweet_bin_labels)):
    value = tweet_bin_labels.iloc[i].safe
    if value == -1.0:
        safe.append(int(2))
    else:
        safe.append(value)

tweet_bin_labels['safe'] = safe

In [8]:
# Check that it all worked
# tweet_bin_labels.head(10)

,time_bin,safe
0,2020-03-23 13:50:00,2
1,2020-03-23 14:05:00,1
2,2020-03-23 14:50:00,2
3,2020-03-23 18:05:00,2
4,2020-03-24 08:35:00,2
5,2020-03-24 10:20:00,2
6,2020-03-24 13:20:00,2
7,2020-03-24 13:50:00,2
8,2020-03-24 16:35:00,2
9,2020-03-24 18:20:00,2


### Preprocess the Current Popularity Data

In [9]:
# curpop.head()

,current_popularity,datetime,binned
0,38,2020-04-02 15:17:00,2020-04-02 15:05:00
1,39,2020-04-02 15:27:00,2020-04-02 15:20:00
2,38,2020-04-02 15:37:00,2020-04-02 15:35:00
3,42,2020-04-02 15:47:00,2020-04-02 15:35:00
4,42,2020-04-02 15:48:00,2020-04-02 15:35:00


In [10]:
# Convert the time bin to datetime
curpop['binned'] = pd.to_datetime(curpop['binned'])
curpop = curpop.reset_index()

In [11]:
curpop = curpop[['current_popularity','binned']]
curpop = curpop.groupby('binned').agg('mean')
curpop = curpop.reset_index()
# curpop

,binned,current_popularity
0,2020-04-02 15:05:00,38.000000
1,2020-04-02 15:20:00,39.000000
2,2020-04-02 15:35:00,40.666667
3,2020-04-02 15:50:00,43.500000
4,2020-04-02 16:05:00,43.000000
...,...,...
441,2020-04-09 16:05:00,16.500000
442,2020-04-09 16:20:00,16.000000
443,2020-04-09 16:35:00,17.000000
444,2020-04-09 16:50:00,21.000000


### Preprocess the Average Time Popularity data

In [12]:
# weekpop.head()

,name,data
0,Monday,"[0, 0, 0, 0, 0, 0, 4, 10, 16, 20, 24, 26, 31, ..."
1,Tuesday,"[1, 0, 0, 0, 0, 2, 7, 12, 18, 23, 27, 30, 32, ..."
2,Wednesday,"[27, 0, 0, 0, 0, 1, 5, 11, 18, 23, 26, 28, 31,..."
3,Thursday,"[1, 0, 0, 0, 0, 1, 6, 13, 21, 27, 28, 28, 29, ..."
4,Friday,"[2, 0, 0, 0, 0, 1, 6, 14, 21, 25, 26, 28, 33, ..."


In [13]:
# lengths = []
# for i in range(0,5):
#    lengths.append(len(weekpop.iloc[i]['data']))
# lengths

[86, 88, 87, 86, 86]

In [14]:
## Ask Huayi about this! Why do the 'data' entries of weekpop have different lengths?

### Preprocess the Image Labels

In [15]:
# img.head()

,timetaken,label
0,4/2/2020 16:05,0
1,4/2/2020 16:50,0
2,4/2/2020 17:50,0
3,4/2/2020 19:05,0
4,4/2/2020 20:05,0


In [16]:
img['timetaken'] = pd.to_datetime(img['timetaken'])
img.columns = ['timetaken','img_label']
# img.head()

,timetaken,img_label
0,2020-04-02 16:05:00,0
1,2020-04-02 16:50:00,0
2,2020-04-02 17:50:00,0
3,2020-04-02 19:05:00,0
4,2020-04-02 20:05:00,0


### Preprocess the Weather Data

In [17]:
#curlog.columns

Index(['time_bin', 'clouds', 'rain_1h', 'snow_1h', 'wind_speed', 'wind_deg',
       'humidity', 'press', 'temp', 'temp_feels', 'temp_max', 'temp_min',
       'park_name', 'sunrise_time', 'sunset_time', 'status',
       'detailed_status'],
      dtype='object')

In [18]:
#wlog.columns

Index(['time_bin', 'time', 'temperature', 'dewpoint', 'humidity',
       'precipitation', 'windspeed', 'winddirection', 'pressure'],
      dtype='object')

In [19]:
# Convert all time columns to datetime
curlog['sunrise_time'] = pd.to_datetime(curlog['sunrise_time'])
curlog['sunset_time'] = pd.to_datetime(curlog['sunset_time'])
curlog['time_bin'] = pd.to_datetime(curlog['time_bin'])
wlog['time_bin'] = pd.to_datetime(wlog['time_bin'])

# Restrict wlog to the dates since the crisis and drop the unnecessary 'time' column
wlog = wlog.set_index('time_bin')
wlog = wlog.sort_index()
wlog = wlog["2020-03-23":]
wlog = wlog.reset_index()
wlog = wlog.drop(['time'], axis = 1)

# To avoid conflicts, drop the 'humidity' column of curlog
curlog = curlog.drop('humidity',axis=1)

In [20]:
#curlog.head()

,time_bin,clouds,rain_1h,snow_1h,wind_speed,wind_deg,press,temp,temp_feels,temp_max,temp_min,park_name,sunrise_time,sunset_time,status,detailed_status
0,2020-04-04 08:35:00,90.0,0.25,0.0,5.10,40.0,1016.0,46.40,38.70,48.200,44.6,Prospect Park,2020-04-04 10:33:33,2020-04-04 23:23:28,Rain,light rain
1,2020-04-04 08:50:00,90.0,0.25,0.0,4.60,40.0,1016.0,46.60,39.56,48.200,44.6,Prospect Park,2020-04-04 10:33:33,2020-04-04 23:23:28,Rain,light rain
2,2020-04-04 09:05:00,90.0,0.00,0.0,4.60,40.0,1016.0,46.69,39.67,48.200,44.6,Prospect Park,2020-04-04 10:33:33,2020-04-04 23:23:28,Rain,light rain
3,2020-04-04 09:20:00,90.0,0.00,0.0,4.35,45.0,1016.0,46.95,40.31,48.595,45.0,Prospect Park,2020-04-04 10:33:33,2020-04-04 23:23:28,Rain,light rain
4,2020-04-04 09:35:00,90.0,0.00,0.0,4.10,50.0,1016.0,47.05,40.75,48.990,45.0,Prospect Park,2020-04-04 10:33:33,2020-04-04 23:23:28,Rain,light rain


In [21]:
#wlog.head()

,time_bin,temperature,dewpoint,humidity,precipitation,windspeed,winddirection,pressure
0,2020-03-23 00:05:00,37.04,-4.575,58.25,0.0,17.575,117.5,1039.2
1,2020-03-23 00:20:00,37.04,-4.750,57.50,0.0,18.550,115.0,1039.3
2,2020-03-23 00:35:00,37.04,-4.925,56.75,0.0,19.525,112.5,1039.4
3,2020-03-23 00:50:00,37.04,-5.100,56.00,0.0,20.500,110.0,1039.5
4,2020-03-23 01:05:00,37.04,-4.825,57.25,0.0,20.500,107.5,1039.4


### Preprocess the Survey Data

In [22]:
#survey.head()

,live,activity,main_track,not_main_track,time_bin
0,No,Walking,1.0,1.0,2020-04-08 18:20:00
1,No,Walking,2.0,0.0,2020-04-04 15:50:00
2,No,Running,1.0,0.0,2020-04-07 18:20:00
3,No,Walking,2.0,1.0,2020-04-07 17:50:00
4,No,Walking,1.0,0.0,2020-04-05 16:50:00


In [23]:
survey['time_bin'] = pd.to_datetime(survey['time_bin'])

In [24]:
# len(survey)

22

### Join our dataframes into one big dataframe

In [25]:
# Index all dataframes by their time bins for easy joining

tweet_bin_labels.set_index('time_bin', inplace = True)
curpop.set_index('binned', inplace = True)
img.set_index('timetaken', inplace = True)
curlog.set_index('time_bin', inplace = True)
wlog.set_index('time_bin', inplace = True)
survey.set_index('time_bin', inplace = True)

In [26]:
df = tweet_bin_labels.join(curpop, how = 'outer')\
.join(img, how = 'outer')\
.join(curlog, how = 'outer')\
.join(wlog, how = 'outer')\
.join(survey, how = 'outer')

In [27]:
# df.head()

In [28]:
# How often is it that 'safe' is null and the survey responses informative?
# df[(pd.notnull(df.safe)) & (pd.notnull(df.main_track)) & (pd.notnull(df.not_main_track))]

In [29]:
# # How often is it unsafe on the main track but safe off of it?
# df[(pd.notnull(df.main_track)) & (pd.notnull(df.not_main_track)) & (df.main_track != df.not_main_track) & (df.main_track != 2.0) & (df.not_main_track != 2.0)]

In [30]:
# Use image label and survey data to improve the 'safe' column
safe = []

# First, let substantive survey data, where it exists, overrule tweets data
for i in range(0, len(df)):
    
    # For timebin i, let:
    #  s be the current safety label
    #  m be the main track's safety
    #  nm be off-main-track safety
    #  im be the image label if it exists
    
    s = df.iloc[i].safe
    m = df.iloc[i].main_track
    nm = df.iloc[i].not_main_track
    im = df.iloc[i].img_label
    
        
    ## Make the survey labels dominate the tweet labels:
        
    # If both m and nm are informative (neither NaN nor 2.0), then:
    if (pd.notnull(m) & pd.notnull(nm) & (m != 2.0) & (nm != 2.0)):
        # append the larger of the two (so: unsafe dominates safe; err on side of caution)
        safe.append(max(m,nm))
        
    # If m is informative but nm is not, append m:
    elif (pd.notnull(m) & (m != 2.0)):
        safe.append(m)
        
    # If nm is informative but m is not, append nm:
    elif (pd.notnull(nm) & (nm != 2.0)):
        safe.append(nm)


    ## If survey values are both null, use image labels. These don't dominate tweets.
        
    # If m and nm are both null, and safe value is 2 (so marked uninformative), and 
    # image label is not null, append image label:
    elif ((s == 2.0) & pd.notnull(im)):
        safe.append(im)
        
    # If m and nm are both null and either s is informative or im is uninformative, append s
    else: 
        safe.append(s)

# Now, replace all NaNs with 2.0 
for i in range(0,len(safe)):
    if pd.isnull(safe[i]):
        safe[i] = 2.0

df['safe'] = safe

In [31]:
#df.head()

,safe,current_popularity,img_label,clouds,rain_1h,snow_1h,wind_speed,wind_deg,press,temp,...,dewpoint,humidity,precipitation,windspeed,winddirection,pressure,live,activity,main_track,not_main_track
2020-03-23 00:05:00,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-4.575,58.25,0.0,17.575,117.5,1039.2,NaN,NaN,NaN,NaN
2020-03-23 00:20:00,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-4.750,57.50,0.0,18.550,115.0,1039.3,NaN,NaN,NaN,NaN
2020-03-23 00:35:00,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-4.925,56.75,0.0,19.525,112.5,1039.4,NaN,NaN,NaN,NaN
2020-03-23 00:50:00,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-5.100,56.00,0.0,20.500,110.0,1039.5,NaN,NaN,NaN,NaN
2020-03-23 01:05:00,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-4.825,57.25,0.0,20.500,107.5,1039.4,NaN,NaN,NaN,NaN


In [32]:
#pd.unique(curlog.detailed_status)

array(['light rain', 'overcast clouds', 'few clouds', 'broken clouds',
       'scattered clouds', 'clear sky', 'moderate rain', 'mist',
       'thunderstorm with rain'], dtype=object)

In [33]:
# Drop unnecessary columns
df = df.drop(['live','activity','status','main_track','not_main_track', 'park_name', 'img_label'],axis=1)

# One-hot encode detailed weather statuses
df = pd.get_dummies(data = df, prefix='', prefix_sep='',columns = ['detailed_status'],dummy_na = True)
# Note: This fills NaNs with 0s-- we will need to fix that

In [34]:
# Where any of the weather columns should be NaN, replace with NaN
statuses = {'light rain':[], 'overcast clouds':[], 'few clouds':[], 'broken clouds':[],
           'scattered clouds':[], 'clear sky':[], 'moderate rain':[], 'mist':[],
           'thunderstorm with rain':[]}

for i in range(0,len(df)):
    if df.iloc[i]['nan'] == 1:
        for status in statuses.keys():
            statuses[status].append(np.NaN)
    else:
        for status in statuses.keys():
            statuses[status].append(df.iloc[i][status])

for status in statuses.keys():
    df[status] = statuses[status]
    
# Drop the now-unnecessary nan column
df = df.drop('nan',axis = 1)

In [35]:
#df.head()

,safe,current_popularity,clouds,rain_1h,snow_1h,wind_speed,wind_deg,press,temp,temp_feels,...,pressure,broken clouds,clear sky,few clouds,light rain,mist,moderate rain,overcast clouds,scattered clouds,thunderstorm with rain
2020-03-23 00:05:00,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1039.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-03-23 00:20:00,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1039.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-03-23 00:35:00,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1039.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-03-23 00:50:00,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1039.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-03-23 01:05:00,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1039.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
#df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1939 entries, 2020-03-23 00:05:00 to 2020-04-14 19:20:00
Data columns (total 30 columns):
safe                      1939 non-null float64
current_popularity        447 non-null float64
clouds                    484 non-null float64
rain_1h                   484 non-null float64
snow_1h                   484 non-null float64
wind_speed                484 non-null float64
wind_deg                  484 non-null float64
press                     484 non-null float64
temp                      484 non-null float64
temp_feels                484 non-null float64
temp_max                  484 non-null float64
temp_min                  484 non-null float64
sunrise_time              484 non-null datetime64[ns]
sunset_time               484 non-null datetime64[ns]
temperature               1917 non-null float64
dewpoint                  1917 non-null float64
humidity                  1917 non-null float64
precipitation             1917 non-null 

In [37]:
# i = 1
# for column in df.columns:
#     if len(pd.unique(df[column])) > 3:
#         print(i, ":", column, ":", len(pd.unique(df[column])))
#         i += 1

1 : current_popularity : 109
2 : clouds : 13
3 : rain_1h : 31
4 : wind_speed : 81
5 : wind_deg : 86
6 : press : 50
7 : temp : 422
8 : temp_feels : 436
9 : temp_max : 75
10 : temp_min : 63
11 : sunrise_time : 8
12 : sunset_time : 8
13 : temperature : 241
14 : dewpoint : 566
15 : humidity : 262
16 : precipitation : 103
17 : windspeed : 248
18 : winddirection : 146
19 : pressure : 808


In [38]:
import matplotlib.pyplot as plt
plt.style.use("ggplot")

fig = plt.figure(figsize=(10,10))

for i in range(0,len(df.columns)):
    if len(pd.unique(df[df.columns[i]])) > 3:
        plt.subplot(5,5,i+1)
        plt.hist(df[df.columns[i]])
        plt.xticks(rotation = 45)
        plt.title(df.columns[i])
#         plt.ylabel("# timebins with that value")
#         plt.xlabel("value for" + column)
plt.tight_layout()
plt.savefig("../src/d06_visuals/predictor_distributions_ee.png")
#plt.show()

/home/eric/anaconda3/lib/python3.7/site-packages/numpy/lib/histograms.py:829: RuntimeWarning: invalid value encountered in greater_equal
  keep = (tmp_a >= first_edge)
/home/eric/anaconda3/lib/python3.7/site-packages/numpy/lib/histograms.py:830: RuntimeWarning: invalid value encountered in less_equal
  keep &= (tmp_a <= last_edge)
/home/eric/anaconda3/lib/python3.7/site-packages/pandas/plotting/_matplotlib/converter.py:103: FutureWarning: Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()
  warnings.warn(msg, FutureWarning)


<Figure size 1000x1000 with 19 Axes>

In [39]:
# Apply a log transform to our exponentially decaying variables

def safe_ln(x):
    if x <= 0:
        return 0
    return np.log(x)

df['rain_1h'] = df['rain_1h'].apply(lambda x: safe_ln(x))
df['precipitation'] = df['precipitation'].apply(lambda x: safe_ln(x))

In [40]:
df.to_pickle(intro+"03_final_multipurpose_df_ee.pkl")